In [1]:
import os
from pybatfish.client.session import Session
import pandas as pd
import yaml
from os.path import abspath, dirname, join, pardir, realpath
from netaddr import *

%run ../code/bgp_route_helpers
%run ../code/gen_external_bgp_adverts

#pandas display options
pd.set_option("display.width", 300)
pd.set_option("display.max_columns", 20)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_colwidth", -1)

#snapshot directory and path setup
_this_dir = os.getcwd()
_root_dir = abspath(join(_this_dir, pardir))

NETWORK = "Nanog-demo1"
BASE_SNAPSHOT_DIR = f"{_root_dir}/snapshots/demo1/base"
BASE_SNAPSHOT_NAME = "base"


# create BF session and initialize snapshot
try:
    bf = Session.get('bfe') #Batfish Enterprise
except:
    bf = Session.get('bf') #Batfish Open-source
bf.set_network(NETWORK)
bf.init_snapshot(BASE_SNAPSHOT_DIR, name=BASE_SNAPSHOT_NAME, overwrite=True)

'base'

# Demo Network
<style>
    .image {
        display: block; 
        margin-left: 20px; 
        margin-right: 20px; 
        width:50%; 
    }
</style>
<img src="./nanog-demo1-network.png" class="image">

**Let's analyze the input routing policy on PE1 towards cust01**

In [2]:
node = 'pe1'
local_ip = '9.1.1.0'
peer_ip = '9.1.1.1'
peer_as = '609'
peer_in_pol = get_bgp_policy_name(bf, node, peer_ip, 'in')

**Retrieve input BGP routes (BGP-Adj-RIB-In) to analyze and convert to format necessary for Batfish to analyze**

In [3]:
input_bgp_routes = get_input_bgp_routes(bf, BASE_SNAPSHOT_DIR, node, peer_ip)

**Analyze behavior of PE1 BGP import policy towards cust01**

In [4]:
df = bf.q.testRoutePolicies(nodes=node, policies=peer_in_pol, direction='in', inputRoutes=input_bgp_routes).answer().frame()

**Display the routes that are denied by the routing policy**

In [5]:
get_denied_routes(df)

,Node,Policy_Name,Input_Route,Action,Output_Route,Difference


**Display the routes that are permitted by the routing policy**

In [6]:
summarize(get_permitted_routes(df)).head(5)

,Node,Network,Action,Difference
0,pe1,11.0.0.0/16,PERMIT,"[{'fieldName': 'communities', 'oldValue': '[609:60001]', 'newValue': '[609:1]'}]"
1,pe1,11.1.0.0/16,PERMIT,"[{'fieldName': 'communities', 'oldValue': '[609:60001]', 'newValue': '[609:1]'}]"
2,pe1,11.2.0.0/16,PERMIT,"[{'fieldName': 'communities', 'oldValue': '[609:60001]', 'newValue': '[609:1]'}]"
3,pe1,11.3.0.0/16,PERMIT,"[{'fieldName': 'communities', 'oldValue': '[609:60001]', 'newValue': '[609:1]'}]"
4,pe1,11.4.0.0/16,PERMIT,"[{'fieldName': 'communities', 'oldValue': '[609:60001]', 'newValue': '[609:1]'}]"


**Display routes that were permitted but not modified**


In [7]:
get_permitted_unmodified(df)

,Node,Policy_Name,Input_Route,Action,Output_Route,Difference


## Change routing policy to block prefixes >/24 and analyze that proposed change

In [8]:
TEST_SNAPSHOT_DIR = f"{_root_dir}/snapshots/demo1/test"
TEST_SNAPSHOT_NAME = "test"

bf.fork_snapshot(BASE_SNAPSHOT_NAME, name=TEST_SNAPSHOT_NAME, add_files=TEST_SNAPSHOT_DIR, overwrite=True)

'test'

### Display the configuration change

In [9]:
# print configuration file difference, this command is ONLY available in the Batfish Enterprise service
try:
    print(bf.get_configuration_diffs(TEST_SNAPSHOT_NAME, BASE_SNAPSHOT_NAME))
except:
    print("This functionality is only available with the Batfish Enterprise service")

This functionality is only available with the Batfish Enterprise service


In [10]:
stream = os.popen(f"icdiff -r --cols=150 {BASE_SNAPSHOT_DIR} {TEST_SNAPSHOT_DIR} -U 2")
print(stream.read())

Only in /Users/samir/git/nanog78/snapshots/demo1/base/configs: mock_cust01.cfg
Only in /Users/samir/git/nanog78/snapshots/demo1/base/configs: mock_cust02.cfg
/Users/samir/git/nanog78/snapshots/demo1/base/configs/pe1.cfg              /Users/samir/git/nanog78/snapshots/demo1/test/configs/pe1.cfg             
ip prefix-list customer1 permit 0.0.0.0/0 le 32                            ip prefix-list customer1 permit 0.0.0.0/0 le 32                           
!                                                                          !                                                                         
                                                                           ip prefix-list BLOCK24 permit 0.0.0.0/0 ge 24                             
                                                                           !                                                                         
ip community-list expanded customer1 permit _609:.*_                       ip community-list

### Compare the old and new routing policy on PE1 towards cust01 (AS 609)

In [11]:
df = compare_bgp_peer_policies(bf, node, peer_ip, BASE_SNAPSHOT_NAME, TEST_SNAPSHOT_NAME, 'in', input_bgp_routes)

**Display routes that were previously permitted but are now denied**

In [12]:
find_newly_denied_routes(df)

,Route,Old Action,Old Transformation,New Action,New Transformation
0,25.0.0.0/26,PERMIT,"[{'fieldName': 'communities', 'oldValue': '[609:60001]', 'newValue': '[609:1]'}]",DENY,None


**Display routes that were previously denied but are not permitted**


In [13]:
find_newly_permitted_routes(df)

,Route,Old Action,Old Transformation,New Action,New Transformation


**Display routes that continued to be permitted, but have attribute changes**


In [14]:
find_routes_with_attrib_change(df)

,Route,Old Action,Old Transformation,New Action,New Transformation


# END